# Section 6 - Persistence

In [ ]:
# Support for MySQL:
# https://pypi.org/project/langgraph-checkpoint-mysql/
# https://pypi.org/project/PyMySQL/
# Article explanation: https://blog.langchain.dev/langgraph-v0-2/

# Get grab execution from where the graph had last left off, i.e. if crashing happening
# Allows for multiple sessions as well.

# Key concepts
# Checkpoints - persistence layer, it will save the state in persistent storage, such as saving to DB
# Save information of state into a DB, like SqLite

In [ ]:
# # Example
# from langgraph.checkpoint.sqlite import SqliteSaver

# # Save memory
# memory = SqliteSaver.from_conn_string(":checkpoints:sqlite:")
# graph = workflow.compile(checkpointer = memory)

# # Local or remote database here to create the connection
# # after each node execption, get user input, get from database, and then

In [ ]:
# Example - Human in the loop

# NO LLM call - the goal is to create the infrsastructure, execute step 1, interrupt from human feedback, then update state

In [7]:
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    input: str
    user_feedback: str


def step_1(state: State) -> None:
    print("---Step 1---")


def human_feedback(state: State) -> None:
    print("---human_feedback---")


def step_3(state: State) -> None:
    print("---Step 3--")


builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# memory options:
# option 1 - use the built-in memory:
memory = MemorySaver()

# # # Fix: Use proper SQLite connection string
# memory = SqliteSaver.from_conn_string("sqlite:///:checkpoints:")

graph = builder.compile(checkpointer=memory, interrupt_before=["human_feedback"])

graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

if __name__ == "__main__":
    thread = {"configurable": {"thread_id": "777"}}

    initial_input = {"input": "hello world"}

    for event in graph.stream(initial_input, thread, stream_mode="values"):
        print(event)

    print(graph.get_state(thread).next)

    user_input = input("Tell me how you want to update the state: ")

    graph.update_state(thread, {"user_feedback": user_input}, as_node="human_feedback")

    print("--State after update--")
    print(graph.get_state(thread))

    print(graph.get_state(thread).next)

    for event in graph.stream(None, thread, stream_mode="values"):
        print(event)

{'input': 'hello world'}
---Step 1---
('human_feedback',)
--State after update--
StateSnapshot(values={'input': 'hello world', 'user_feedback': 'cheese'}, next=('step_3',), config={'configurable': {'thread_id': '777', 'checkpoint_ns': '', 'checkpoint_id': '1efc97fe-dd81-636e-8002-a6d02ecbe4de'}}, metadata={'source': 'update', 'writes': {'human_feedback': {'user_feedback': 'cheese'}}, 'thread_id': '777', 'step': 2, 'parents': {}}, created_at='2025-01-03T03:07:56.016497+00:00', parent_config={'configurable': {'thread_id': '777', 'checkpoint_ns': '', 'checkpoint_id': '1efc97fe-c5bf-6622-8001-4c54eb750d3b'}}, tasks=(PregelTask(id='8a503306-8b22-2bb1-bce5-9b04a1ae2ef9', name='step_3', path=('__pregel_pull', 'step_3'), error=None, interrupts=(), state=None, result=None),))
('step_3',)
{'input': 'hello world', 'user_feedback': 'cheese'}
---Step 3--


# TL;DR - Persistence in LangGraph

🔑 Key Concepts
1. State Management: Track and maintain graph state 
2. Checkpointing: Save progress for recovery/persistence

* Interruptions: Pause for human input/validation
* Threading: Track separate graph executions
* Streaming: Control and monitor graph execution flow

💡 Best Practices
* Use get_state() to inspect graph state
* Check next node before executing
* Use checkpointers for production applications
* Handle interruptions for interactive flows
* Use thread IDs to manage multiple executions


This pattern is especially useful for:
Interactive AI applications <br>
Multi-step processes <br>
Applications requiring human feedback <br>
Stateful workflows <br>
Async operations <br>

In [ ]:
# Example Code:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver

# --- 1. Define State Structure ---
class State(TypedDict):
    input: str
    user_feedback: str

# --- 2. Define Node Functions ---
def step_1(state: State) -> None:
    print("Processing input...")
    return {"processed": state["input"]}

def human_feedback(state: State) -> None:
    return {"feedback_processed": state["user_feedback"]}

def step_3(state: State) -> None:
    return {"final_result": "completed"}

# --- 3. Build Graph ---
builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)

# Add edges
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# --- 4. Setup Checkpointing ---
# Option 1: In-Memory
memory = MemorySaver()
# Option 2: Persistent Storage
# memory = SqliteSaver.from_conn_string("sqlite:///checkpoints.db")

# --- 5. Compile Graph ---
graph = builder.compile(
    checkpointer=memory,
    interrupt_before=["human_feedback"]
)

# --- 6. Execute Graph ---
if __name__ == "__main__":
    # Initialize thread and input
    thread = {"configurable": {"thread_id": "123"}}
    initial_input = {"input": "hello world"}

    # Start execution
    print("Starting execution...")
    for event in graph.stream(initial_input, thread, stream_mode="values"):
        print(event)

    # Check next node
    state = graph.get_state(thread)
    print(f"Next node: {state.next}")

    # Get human input
    user_input = input("Enter feedback: ")
    
    # Update state with feedback
    graph.update_state(
        thread, 
        {"user_feedback": user_input}, 
        as_node="human_feedback"
    )

    # Check state after update
    print("State after update:", graph.get_state(thread))

    # Continue execution
    print("Resuming execution...")
    for event in graph.stream(None, thread, stream_mode="values"):
        print(event)